In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

Saving sales_data_cleaned.csv to sales_data_cleaned.csv
User uploaded file "sales_data_cleaned.csv" with length 553653 bytes


In [2]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['sales_data_cleaned.csv']))

display(df.head())

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,897 Long Airport Avenue,Unknown,NYC,NY,10022,USA,Unknown,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,59 rue de l'Abbaye,Unknown,Reims,Unknown,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,Unknown,Paris,Unknown,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,78934 Hillside Dr.,Unknown,Pasadena,CA,90003,USA,Unknown,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,7734 Strong St.,Unknown,San Francisco,CA,Unknown,USA,Unknown,Brown,Julie,Medium


In [5]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Load the dataset
# Ensure the file 'sales_data_cleaned.csv' is uploaded to Colab files
try:
    df = pd.read_csv('sales_data_cleaned.csv')
    df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
    print("Success: Data loaded correctly.")
except Exception as e:
    print(f"Error: {e}")

# 2. Prepare the Dashboard Layout (2 rows x 2 columns)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Monthly Sales Performance", "Revenue by Product Line",
                    "Top 10 Countries by Sales", "Sales by Deal Size"),
    specs=[[{"type": "xy"}, {"type": "domain"}],
           [{"type": "bar"}, {"type": "bar"}]]
)

# --- Chart 1: Monthly Sales Trend ---
monthly = df.groupby(['YEAR_ID', 'MONTH_ID'])['SALES'].sum().reset_index()
for year in monthly['YEAR_ID'].unique():
    year_data = monthly[monthly['YEAR_ID'] == year]
    fig.add_trace(go.Scatter(x=year_data['MONTH_ID'], y=year_data['SALES'],
                             name=f"Year {year}", mode='lines+markers'), row=1, col=1)

# --- Chart 2: Product Line Distribution (Donut) ---
prod_sales = df.groupby('PRODUCTLINE')['SALES'].sum().reset_index()
fig.add_trace(go.Pie(labels=prod_sales['PRODUCTLINE'], values=prod_sales['SALES'],
                     hole=.4, name="Products"), row=1, col=2)

# --- Chart 3: Top 10 Countries (Horizontal Bar) ---
country_sales = df.groupby('COUNTRY')['SALES'].sum().nlargest(10).reset_index()
fig.add_trace(go.Bar(x=country_sales['SALES'], y=country_sales['COUNTRY'],
                     orientation='h', marker=dict(color='#3366CC'), name="Countries"), row=2, col=1)

# --- Chart 4: Deal Size Analysis (Bar) ---
deal_sales = df.groupby('DEALSIZE')['SALES'].sum().reset_index()
fig.add_trace(go.Bar(x=deal_sales['DEALSIZE'], y=deal_sales['SALES'],
                     marker=dict(color='#109618'), name="Deal Size"), row=2, col=2)

# 3. Apply Professional Styling (Dark Theme)
fig.update_layout(
    height=900,
    title_text="Global Sales Executive Dashboard",
    template="plotly_dark",
    showlegend=True,
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e",
    font=dict(family="Arial", size=12)
)

# Update axis labels
fig.update_xaxes(title_text="Month", row=1, col=1)
fig.update_yaxes(title_text="Sales ($)", row=1, col=1)

# Show final Dashboard
fig.show()

Success: Data loaded correctly.
